## Para analizar y limpiar datos del JSON Growth

In [ ]:
import json
import pandas as pd
import plotly.express as px

: 

In [ ]:
import json

with open('data_ciabatta.json', 'r') as f:
    datos_ciabatta = json.load(f)

with open('data_vicenta.json', 'r') as f:
    datos_vicenta = json.load(f)

# Merge the data
datos_merged = datos_ciabatta + datos_vicenta

# Save the merged data to a new JSON file
new_filename = 'nuevo.json'
with open(new_filename, 'w') as f:
    json.dump(datos_merged, f)

: 

In [ ]:
df = pd.read_json('nuevo.json')

: 

In [ ]:
df.to_excel('GRW.xlsx', index=False)

: 

In [ ]:
import pandas as pd
import json

# Lee el archivo XLSX y obtén los datos de la columna
df = pd.read_excel('GRW.xlsx')
columna = df['detail']

# Analiza las cadenas JSON y crea una lista de diccionarios
data = []
for item in columna:
    diccionarios = json.loads(item.replace("'", '"'))
    for diccionario in diccionarios.values():
        data.extend(diccionario)

# Crea un DataFrame a partir de la lista de diccionarios
df_resultado = pd.DataFrame(data)

# Obtiene la información del DataFrame
filas_totales = df_resultado.shape[0]
columnas_totales = df_resultado.shape[1]

# Imprime la información del DataFrame
print("Filas totales:", filas_totales)
print("Columnas totales:", columnas_totales)


: 

In [ ]:
# Convierte la columna de fecha a formato datetime si no está en ese formato
df_resultado['month'] = pd.to_datetime(df_resultado['month'])

# Ordena el DataFrame por la columna de fecha en orden ascendente
df_resultado = df_resultado.sort_values('month')

# Obtiene la primera fecha del DataFrame
primera_fecha = df_resultado['month'].iloc[0]

print("La primera fecha en el conjunto de datos es:", primera_fecha)

: 

In [ ]:
df_resultado = df_resultado.drop(['cityName', 'country', 'brandId'], axis=1)

: 

In [ ]:
num_filas_duplicadas = df_resultado.duplicated().sum()
print("Número de filas duplicadas: ", num_filas_duplicadas)

: 

In [ ]:
df_resultado.to_excel('OperadoresGRW.xlsx', index=False)

: 

## Para extraer data de Platforms de Index 

In [ ]:
import awswrangler as wr
import pandas as pd
import boto3
import datetime
import json
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

: 

In [ ]:
L1_OPERATIONS_BUCKET = 'l1-operations'
L1_OPERATIONS_PREFIX = 'platforms-tables/last-snapshot/'
L1_OPERATIONS_STORES = 'platforms_store.csv.gz'
L1_OPERATIONS_BRANCHES = 'platforms_branch_full.csv.gz'
L1_OPERATIONS_BRANDS = 'platforms_brand.csv.gz'

# *Sales Channels
L1_BUCKET = 'l1-sales-channels'


def get_stores():
    """
        Read L1 data tables of operations app.
        
        Returns:
            df: DataFrame
    """
    s3_bucket = L1_OPERATIONS_BUCKET
    s3_prefix = L1_OPERATIONS_PREFIX.strip("/")
    s3_stores = L1_OPERATIONS_STORES
    path = f"s3://{s3_bucket}/{s3_prefix}/{s3_stores}"

    df_stores = wr.s3.read_csv(path=path, dataset=False)
    
    df_stores = (df_stores.rename(columns={
                                 "start_date": "store_start_date",
                                 "end_date": "store_end_date",
                                 "is_integrated": "store_is_integrated"}))
    df_stores["app_store_id"] = df_stores.app_store_id.astype(str)

    return df_stores


def get_branches():
    """
        Read L1 data tables of operations app.
        
        Returns:
            df: DataFrame
    """
    s3_bucket = L1_OPERATIONS_BUCKET
    s3_prefix = L1_OPERATIONS_PREFIX.strip("/")
    s3_branches = L1_OPERATIONS_BRANCHES
    path = f"s3://{s3_bucket}/{s3_prefix}/{s3_branches}"
    df_branches = wr.s3.read_csv(path=path, dataset=False)
    
    return df_branches
    

def get_brands():
    """
        Read L1 data tables of operations app.
        Returns:
            df: DataFrame
    """
    s3_bucket = L1_OPERATIONS_BUCKET
    s3_prefix = L1_OPERATIONS_PREFIX.strip("/")
    s3_brands = L1_OPERATIONS_BRANDS
    path = f"s3://{s3_bucket}/{s3_prefix}/{s3_brands}"

    df_brands = wr.s3.read_csv(path=path, dataset=False)
    df_brands = df_brands[["id", "name"]].rename(columns={"id": "brand_id", "name": "brand_name"})
    
    return df_brands


def get_platforms_data():
    """
        Merge tables of operations app.
        Returns:
            df: DataFrame
    """
    try:
        # Brands Definition
        df_brands = get_brands()
        df_brands = df_brands[['brand_id', 'brand_name']]

        # Branches Definition
        df_branches = get_branches()
        df_branches = df_branches[['id', 'name', 'address', 'is_own', 'operator_id', 'operator_name', 'lat', 'lng', 'city_country']]

        # Stores Definition
        df_stores = get_stores()
        df_stores = df_stores[['branch_id', 'brand_id', 'app', 'app_store_id', 'is_active', 'store_start_date', 'store_end_date']]

        # Merge Stores & Brands
        df_stores_brands = pd.merge(df_stores, df_brands, how="left", left_on='brand_id', right_on='brand_id')
        # Add Branches Information
        df = pd.merge(df_stores_brands, df_branches, how="left", left_on='branch_id', right_on='id')

        # Selection of Columns
        df = df[['app_store_id', 'branch_id', 'name', 'brand_id', 'brand_name', 'app', 
                 'store_start_date', 'store_end_date', 'is_active', 
                 'is_own', 'operator_id', 'operator_name',
                 'address', 'lat', 'lng', 'city_country']]
        df = df.rename(columns={'name': 'branch_name', 'city_country': 'country'})

        # Create MARGIN store_start_date to be use in order logic
        for index, row in df.iterrows():
            try:
                # If store_id was used before...
                idx =  df[(df['app_store_id'] == row['app_store_id']) & 
                          (df['store_start_date'] != row['store_start_date']) & 
                          (df['store_end_date'] < row['store_start_date'])].index.values.astype(int)[0]

                df.loc[index, 'store_margin_start_date'] = df.at[idx,'store_end_date']
            except:
                pass
            
        df['store_start_date'] = pd.to_datetime(df['store_start_date'])
        df['store_margin_start_date'] = pd.to_datetime(df['store_margin_start_date'])
        df['store_margin_start_date'] = np.where(df['store_margin_start_date'].isnull(), 
                                                 df['store_start_date'] - datetime.timedelta(days=90), 
                                                 df['store_margin_start_date'])
        
        # Restore to default type
        df['store_start_date'] = df['store_start_date'].dt.strftime('%Y-%m-%d')
        df['store_margin_start_date'] = df['store_margin_start_date'].dt.strftime('%Y-%m-%d')
        
        # Fill Franchise Features
        df["operator_name"].fillna("Kitchenita", inplace = True)

        return { "statusCode": 200, "df": df }
    except Exception as err:
        return { "statusCode": 500, "body": f"Failed while trying to get Platforms DATA: {err}" }

: 

In [ ]:
dresponse  = get_platforms_data()
dfStores   = dresponse['df']

dfStoresRP = dfStores[(dfStores['app'] == 'RP')]

dfStoresRP['app_store_id'] = dfStoresRP['app_store_id'].astype(int)

dfStoresRP

: 

In [ ]:
country = "AR"
city = "BUE"

from_date = datetime.date(2023,1,15)
to_date = datetime.date(2023,7,5)

df_index = pd.DataFrame()
prefix = "s3://l2-food-crawlers/rappi/index"
for days_offset in range((to_date - from_date).days + 1):
    date = from_date + datetime.timedelta(days=days_offset)
    path = f"{prefix}/country={country}/city={city}/year={date.year:04d}/month={date.month:02d}/day={date.day:02d}"
    
    # Si no están todos los días...
    try:
        df_date_index = wr.s3.read_parquet(path, dataset=True)

        df_date_index = pd.merge(df_date_index, dfStoresRP, left_on="store_id", right_on='app_store_id')
        #df_index = df_index.append(df_date_index, ignore_index=True)
        df_index = pd.concat([df_index, df_date_index])
    except Exception as err:
        print(f"No encontró {date}")

: 

In [ ]:
mask_lunch  = (datetime.time(8) < df_index["local_time"]) &  (datetime.time(17) > df_index["local_time"])
mask_dinner = (datetime.time(19) < df_index["local_time"]) &  (datetime.time(23,59,59) > df_index["local_time"])
df_index["slot_time"] = None

df_index.loc[mask_dinner, "slot_time"] = "dinner"
df_index.loc[mask_lunch,  "slot_time"] = "lunch"

: 

In [ ]:
df_index

: 

In [ ]:
df_index.to_excel('OperadoresIND.xlsx', index=False)

: 

## Para extraer data de Platforms de OPS 

In [ ]:
# Esta es la libreria que nos permite levantar datos de AWS como dataframes de Pandas
import awswrangler as wr

import pandas as pd
import numpy as np

import datetime

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

: 

In [ ]:
# Leemos datos del operador (calculo diario)
df_daily = wr.s3.read_parquet("s3://data-prod-l2-metrics/internal_use/operators/daily/snapshot/", dataset=True)

# Leemos los datos de 
df_calendar_month = wr.s3.read_csv("s3://data-prod-l2-metrics/internal_use/operators/calendar_month/snapshot/", dataset=True)

# Convertimos en formato fecha a la columna fecha
df_daily["date"] = pd.to_datetime(df_daily["date"])

df_calendar_month["day"] = 1
df_calendar_month["date"] = pd.to_datetime(df_calendar_month[["year", "month", "day"]])

# Rellenamos una columna
df_daily["app"] = df_daily["app"].fillna("All")
df_calendar_month["app"] = df_calendar_month["app"].fillna("All")

# Ordenamos los datos con las fechas (de mayor a menor), pais, marca y cocina.
df_daily.sort_values(by=["date", "country", "brand_id", "branch_id"], ascending=[False, True, True, True], inplace=True)
df_calendar_month.sort_values(by=["date", "country", "brand_id", "branch_id"], ascending=[False, True, True, True], inplace=True)

# Corregimos algunas inconsistencias por cambios
df_daily["is_in_warning"] = df_daily["is_in_warning"].fillna(df_daily["is_regular"])
df_daily["is_regular"] = df_daily["is_over_performing"].fillna(df_daily["is_regular"])
df_daily.drop(columns=["is_over_performing"], inplace=True)

df_daily["buy_back_percent"] = df_daily["buy_back_percent"].fillna(0)
df_calendar_month["buy_back_percent"] = df_calendar_month["buy_back_percent"].fillna(0)

: 

In [ ]:
df_daily

: 

In [ ]:
df_daily.to_excel('OperadoresOPS.xlsx', index=False)

: 

## Juntar todos los dataset

In [ ]:
import pandas as pd
import numpy as np

: 

In [ ]:
# Read the Excel file
OPS = pd.read_excel("OperadoresOPS.xlsx")

: 

In [ ]:
OPS = OPS[(OPS['branch_name'] == 'Palermo - Santa Fe') |
          (OPS['branch_name'] == 'Martinez - Av Santa Fe') |
          (OPS['branch_name'] == 'Almagro - Rivadavia') |
          (OPS['branch_name'] == 'Monte Castro - Lope de Vega') |
          (OPS['branch_name'] == 'Flores - Av Eva Perón')]

OPS = OPS[OPS['brand_name'] == 'Ciabatta']

OPS = OPS[OPS['app'] == 'RP']


: 

In [ ]:
OPS = OPS.sort_values(by='date')

: 

In [ ]:
OPS = OPS.dropna(axis=1, how='all')

: 

In [ ]:
column_names = OPS.columns.tolist()

# Display the column names
print(column_names)

: 

In [ ]:
columns_to_drop = ['country', 'branch_id', 'brand_id', 'year', 'month', 'trunk_id', 'trunk_name', 'company_id', 'app','city_id', 'city_name']

# Drop the specified columns from the DataFrame
OPS = OPS.drop(columns=columns_to_drop)

: 

In [ ]:
OPS['date'] = pd.to_datetime(OPS['date'])

OPS['month'] = OPS['date'].dt.to_period('M').dt.to_timestamp()

df_median_ops = OPS.groupby(['branch_name', 'month']).median()

df_median_ops = df_median_ops.reset_index()

OPS = df_median_ops.sort_values(['branch_name', 'month'])

OPS['month'] = OPS['month'].dt.strftime('%Y-%m-%d')


: 

In [ ]:
# Read the Excel file
IND = pd.read_excel("operadoresIND.xlsx")

: 

In [ ]:
IND = IND[(IND['branch_name'] == 'Palermo - Santa Fe') |
          (IND['branch_name'] == 'Martinez - Av Santa Fe') |
          (IND['branch_name'] == 'Almagro - Rivadavia') |
          (IND['branch_name'] == 'Monte Castro - Lope de Vega') |
          (IND['branch_name'] == 'Flores - Av Eva Perón')]

IND = IND[IND['brand_name'] == 'Ciabatta']

IND = IND[IND['app'] == 'RP']


: 

In [ ]:
# Convert the "local_date" column to datetime format
IND['local_date'] = pd.to_datetime(IND['local_date'])

# Extract the year and month components from the "local_date" column
IND['year'] = IND['local_date'].dt.year
IND['month'] = IND['local_date'].dt.month

# Create the "month" column with the desired format
IND['month'] = pd.to_datetime(IND[['year', 'month']].assign(day=1))

: 

In [ ]:
df_median = IND.groupby(['branch_name', 'month']).median()

df_median = df_median.reset_index()

IND = df_median.sort_values(['branch_name', 'month'])

: 

In [ ]:
IND = IND[['branch_name', 'month', 'index_position', 'store_id']]

IND['store_id'] = IND['store_id'].astype(int)

: 

In [ ]:
DICT = IND[['store_id', 'branch_name']].copy()

: 

In [ ]:
# Read the Excel file
GRW = pd.read_excel("OperadoresGRW.xlsx")

: 

In [ ]:
GRW = GRW[(GRW['storeId'] == 212406)|
          (GRW['storeId'] == 202549)|
          (GRW['storeId'] == 210064)|
          (GRW['storeId'] == 212443) |
          (GRW['storeId'] == 207898) ]


: 

In [ ]:
GRW = pd.merge(GRW, DICT, left_on='storeId', right_on='store_id', how='left')

: 

In [ ]:
# Convert 'date' column to datetime format
GRW['month'] = pd.to_datetime(GRW['month'])

# Add a 'month' column with the first day of each month
GRW['month'] = GRW['month'].dt.to_period('M').dt.to_timestamp()

# Group by 'branch_name' and 'month' and calculate the median
df_median_grw = GRW.groupby(['branch_name', 'month']).median()

# Reset the index
df_median_grw = df_median_grw.reset_index()

# Sort by 'branch_name' and 'month'
df_median_grw = df_median_grw.sort_values(['branch_name', 'month'])

# Format 'month' column as a string in 'YYYY-MM-DD' format
df_median_grw['month'] = df_median_grw['month'].dt.strftime('%Y-%m-%d')

# Update the original DataFrame 'GRW' with the calculated values
GRW = df_median_grw

: 